In [61]:
import networkx as nx
import numpy as np
from numpy  import *
import math
import pandas as pd
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import statistics

In [5]:
def degree_distribution(GER):
    vk = dict(GER.degree())
    vk = list(vk.values()) 
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= arange(0,maxk+1) 
    Pk = np.zeros(maxk+1) 
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) 
    return kvalues,Pk

In [6]:
def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

In [7]:
def average_clustering (self):
  avc = nx.average_clustering(self)
  return avc

In [8]:
def average_degree(self):
  N = len(self)
  M = self.number_of_edges()
  return 2*M/N

In [9]:
def random_graphs(N, p):
  return nx.gnp_random_graph(N, p)

In [10]:
def random_graphs_small(N, av_degree, p):
  return nx.watts_strogatz_graph(N, av_degree, p, seed=None)

In [11]:
def distances(self):
    N = len(self)
    vl = []
    for i in np.arange(0, N):
        for j in np.arange(i+1, N):
            if(i != j):
                aux = nx.shortest_path(self,i,j)
                vl.append(len(aux)-1)
    return vl

def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*math.log(p, 2)
    return H

**Exercicio 1:**

Calcule o coeficiente aglomeração e segundo momento do grau para uma rede BA com grau médio igual a 10 e N=1000.

In [117]:
N = 1000
av_degree = 10
i = 0
list_momments = []
list_avg = []
while (i < 10):
  m = int(av_degree/2)
  G = nx.barabasi_albert_graph(N, m)
  Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
  G = G.subgraph(Gcc[0])
  avg_c = np.mean(list(nx.clustering(G).values()))
  momment = momment_of_degree_distribution(G, 2)
  list_avg.append(avg_c)
  list_momments.append(momment)
  i += 1

avg_momments = statistics.mean(list_momments)
avg_c = statistics.mean(list_avg)
print("AVG: %.3f"%avg_c, "Momment: %.f"%avg_momments)

AVG: 0.040 Momment: 202


**Exercicio 2:**
Considere uma rede aleatória (Erdos-Renyi) e uma rede BA com N=1000 vértices e grau médio 10. Qual o valor da entropia de Shannon da distribuição do grau para essas redes?

In [21]:
N = 1000
av_degree = 10

p_erdos = av_degree/(N-1)
G_erdos = nx.erdos_renyi_graph(N, p_erdos)
H_erdos = shannon_entropy(G_erdos)

m_barabasi = int(av_degree/2)
G_barabasi = nx.barabasi_albert_graph(N, m_barabasi)
H_barabasi = shannon_entropy(G_barabasi)

print("Erdos-Renyi: %.1f"%H_erdos, "| Barabási-Albert: %.1f"%H_barabasi)

Erdos-Renyi: 3.7 | Barabási-Albert: 3.6


**Exercicio 3:**

Considere o modelo de Barabási-Albert com N=1000 e grau médio igual a 10. Calcule o coeficiente de correlação entre o grau e a medida eigenvector centrality. 

In [40]:
N = 1000
av_degree = 10

m_barabasi = av_degree // 2
G_barabasi = nx.barabasi_albert_graph(N, m_barabasi)
dict_degree = dict(G_barabasi.degree())
dict_eigenvector_c = nx.eigenvector_centrality(G_barabasi)
df_degree = pd.DataFrame.from_dict(dict_degree, orient="index", columns=["degree"]).reset_index()
df_eigenvector_c = pd.DataFrame.from_dict(dict_eigenvector_c, orient="index", columns=["eigen"]).reset_index()

df = df_degree.merge(df_eigenvector_c, on="index", how="inner")

rho, _ = pearsonr(df.degree, df.eigen)

print("Pearson Coefficient: {:.1f}".format(rho))

Pearson Coefficient: 0.9


**Exercicio 4:**

Calcule a correlação entre a medida betweeness centrality e o grau para uma rede BA. Considere N=500 e grau médio 10.

In [53]:
N = 500
av_degree = 10

m_barabasi = int(av_degree/2)
G_barabasi = nx.barabasi_albert_graph(N, m_barabasi)

dict_degree = dict(nx.degree(G_barabasi))
dict_betweenness_c = nx.betweenness_centrality(G_barabasi)

df_degree = pd.DataFrame.from_dict(dict_degree, orient = "index", columns = ["degree"]).reset_index()
df_betweenness_c = pd.DataFrame.from_dict(dict_betweenness_c, orient = "index", columns = ["betweenness"]).reset_index()

df = df_degree.merge(df_betweenness_c, on = "index", how = "inner")

rho, _ = pearsonr(df.degree, df.betweenness)
corr, _ = spearmanr(df.degree, df.betweenness)

print("Pearson Coefficient: %.2f"%rho)

Pearson Coefficient: 0.96


**Exercicio 5:**

Calcule o segundo momento do grau para o modelo de configuração com a=3. Considere N=500. Selecione o valor mais próximo, pois pode haver alguma variação devido à natureza estocástica da simulação. Use:
G0=nx.configuration_model(seq)

Gcc = sorted(nx.connected_components(G0), key=len, reverse=True)

G = G0.subgraph(Gcc[0])

In [107]:
from random import *
np.random.seed()
N = 500
a = 3
list_momments = []

for i in range(10):
  seq = np.random.zipf(a, N) 
  if(sum(seq)%2 != 0): 
      pos = randint(0, len(seq))
      seq[pos] = seq[pos]+ 1

  G=nx.configuration_model(seq)
  Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
  G = G.subgraph(Gcc[0])
  momment = momment_of_degree_distribution(G, 2)
  list_momments.append(momment)

avg_momments = statistics.mean(list_momments)
print("Momment: %.1f"%avg_momments)

Momment: 12.5
